## Instrucciones:

Este archivo es una plantilla que deberás completar para la creación de un job que permita ejecutar tu experimento asignado. Es importante reemplazar el nombre genérico del archivo por un identificador único, que será el número asignado al alumno que lo creó. Cada vez que otro alumno utilice ese job, se recompensará al autor original.

El job debe ser ejecutable simplemente configurando correctamente las siguientes secciones:
- **Parámetros**
- **Input**
- **Output**

El archivo debe ser obligatoriamente un *Jupyter Notebook*, con el fin de facilitar su uso a los alumnos menos técnicos. Sin embargo, si prefieres migrar o adaptar el código a archivos planos para tus propios pipelines, podrás hacerlo, siempre que quede claro el origen del código.

El formato del nombre del job debe seguir la siguiente convención:
- `j<id_alumno>_<ref>.ipynb`

Cada alumno puede generar más de un job, y también puede modificar los jobs de otros, siempre que se respete el nombre original.

Para la entrega de la segunda competencia, debes incluir dos componentes:
1. Todos los jobs utilizados, con sus nombres originales.
2. Un archivo explicando la secuencia en la que se ejecutaron los jobs.

Los jobs se compartirán en las slides de los experimentos con el link al repo de github.
---


# Título: Ajuste por inflacion (IPC)

## Autor: Eugenio Negrin
## Fecha de última modificación: 6/11/2024
## Descripción:

Transforma las variables monetarias de pesos corrientes a pesos constantes (en una base específica) con el objetivo de hacer comparables los valores monetarios a lo largo del tiempo. Para ello, el script brinda la opcion de utilizar 2 indices de precios alternativos:

- El Índice de Precios al Consumidor (IPC) elaborado por el INDEC ([ver acá](https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-5-31)), siendo este índice el más usado para el cálculo de la inflación en Argentina.
- El Indice que elabora UCEMA ([ver acá](https://ucema.edu.ar/cea/canasta)) a partir de la Canasta del Profesional Ejecutivo (CPE), que busca estimar el gasto promedio del grupo familiar representativo de un profesional ejecutivo residente en el area metropolitana del Gran Buenos Aires. 

El procedimiento consiste en dividir el valor nominal o corriente de una variable por el coeficiente de deflactación, obteniendo así el valor real o en pesos constantes:

`valor real = valor nominal / coeficiente de deflactación`

El coeficiente de deflactación (o deflactor) para el mes t es:

`coeficiente de deflactación en el mes t = IPC_t / IPC_base`

El mes que tomamos como base es arbitrario y dependerá de respecto a qué mes queremos expresar nuestras variables. Por ejemplo, si deseamos que los valores monetarios estén expresados en pesos de agosto de 2020, debemos usar ese mes como base. De esta forma, todas las variables monetarias estarán expresadas en pesos equivalentes al poder de compra que tenía la moneda en agosto de 2020.

Es importante notar que, luego de aplicar el proceso, las variables serán comparables independientemente del mes que se use como base, afectando esto solamente la magnitud de los valores, pero no su comparabilidad.

Puede encontrar una descripción mas detallada del procedimiento de ajuste por inflación en el siguiente link: https://economipedia.com/definiciones/deflactor.html

## Parámetros

**indice**: Indice de precios que se utilizara para realizar el ajuste por inflacion, puede tomar los valores 'ipc' o 'cpe'

**periodo_base**: mes que tomamos como referencia, las variables monetarias estaran expresadas en pesos constantes equivalentes al poder adquisitivo de la moneda en ese mes. El IPC publicado por el INDEC usa como periodo base a Diciembre 2016 (IPC Dic 2016 = 100), pero se puede elegir otro periodo, en particular periodos que incluyamos en nuestra base. Si desea mantener la base original asigne -1 a este parametro

**sufijo**: El script permite definir un sufijo para las variables ajustadas en el caso de que no querramos sobreescrbir las originales, si este parametro se deja vacio ("") el script sobreescribe las variables originales.

**monetarias**: lista de variables monetarias a ajustar, por defecto tomo las 73 variables monetarias del dataset original pero se pueden sacar o agregar nuevas si asi lo desea.



In [2]:
indice = 'ipc' # ipc o cpe

periodo_base = 202101
# periodo_base = -1 # si desea mantener la base original (IPC Dic 2016 = 100)

sufijo = "" # si es = a "" modifica las variables originales
# sufijo = "_adj" # Si no quiere sobreescribir agregue un sufijo

# Lista de variables monetarias a ajustar
monetarias = [
    "mrentabilidad"
    , "mrentabilidad_annual"
    , "mcomisiones"
    , "mactivos_margen"
    , "mpasivos_margen"
    , "mcuenta_corriente_adicional"
    , "mcuenta_corriente"
    , "mcaja_ahorro"
    , "mcaja_ahorro_adicional"
    , "mcaja_ahorro_dolares"
    , "mcuentas_saldo"
    , "mautoservicio"
    , "mtarjeta_visa_consumo"
    , "mtarjeta_master_consumo"
    , "mprestamos_personales"
    , "mprestamos_prendarios"
    , "mprestamos_hipotecarios"
    , "mplazo_fijo_dolares"
    , "mplazo_fijo_pesos"
    , "minversion1_pesos"
    , "minversion1_dolares"
    , "minversion2"
    , "mpayroll"
    , "mpayroll2"
    , "mcuenta_debitos_automaticos"
    , "mttarjeta_visa_debitos_automaticos"
    , "mttarjeta_master_debitos_automaticos"
    , "mpagodeservicios"
    , "mpagomiscuentas"
    , "mcajeros_propios_descuentos"
    , "mtarjeta_visa_descuentos"
    , "mtarjeta_master_descuentos"
    , "mcomisiones_mantenimiento"
    , "mcomisiones_otras"
    , "mforex_buy"
    , "mforex_sell"
    , "mtransferencias_recibidas"
    , "mtransferencias_emitidas"
    , "mextraccion_autoservicio"
    , "mcheques_depositados"
    , "mcheques_emitidos"
    , "mcheques_depositados_rechazados"
    , "mcheques_emitidos_rechazados"
    , "matm"
    , "matm_other"
    , "Master_mfinanciacion_limite"
    , "Master_msaldototal"
    , "Master_msaldopesos"
    , "Master_msaldodolares"
    , "Master_mconsumospesos"
    , "Master_mconsumosdolares"
    , "Master_mlimitecompra"
    , "Master_madelantopesos"
    , "Master_madelantodolares"
    , "Master_mpagado"
    , "Master_mpagospesos"
    , "Master_mpagosdolares"
    , "Master_mconsumototal"
    , "Master_mpagominimo"
    , "Visa_mfinanciacion_limite"
    , "Visa_msaldototal"
    , "Visa_msaldopesos"
    , "Visa_msaldodolares"
    , "Visa_mconsumospesos"
    , "Visa_mconsumosdolares"
    , "Visa_mlimitecompra"
    , "Visa_madelantopesos"
    , "Visa_madelantodolares"
    , "Visa_mpagado"
    , "Visa_mpagospesos"
    , "Visa_mpagosdolares"
    , "Visa_mconsumototal"
    , "Visa_mpagominimo" 
]

## Input

< Archivos de datos (csv o csv.gz) con sus paths que van a consumirse por el job>

In [3]:
# base_path = '/home/eanegrin/buckets/b1/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'

dataset_path = base_path + 'datasets/'

dataset_file = 'competencia_02.parquet'
indices_file = 'inflacion_ipc_cpe.csv' # debe estar la carpeta espcificada en dataset_path

## Output

< Archivos que va a generar el job>

In [4]:
# Por defecto le agrega _ipcadj al nombre del archivo original
base = dataset_file[:dataset_file.find('.')] 
extension = dataset_file[dataset_file.find('.'):] 
output_file = f"{base}_inflacion_adj{extension}"

# si se prefiere otro nombre:
# output_file = "output_ejemplo.csv"

## Procesos

### Paquetes necesarios

In [5]:
#!pip install polars

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [5]:
import polars as pl

df = pl.read_parquet(dataset_path + dataset_file)
df_inflacion = pl.read_csv(dataset_path + indices_file)

In [ ]:
df_inflacion = df_inflacion.select([
    pl.col("periodo"),
    pl.col(indice).alias("indice")
])

In [10]:
# se convierte el IPC al mes base especificado
if periodo_base == -1:
    base = 100
else:
    base = df_inflacion.filter(pl.col("periodo") == periodo_base)["indice"][0]
    
df_inflacion = df_inflacion.with_columns((pl.col("indice") / base).alias("deflactor"))

In [11]:
# Agregamos la columna con el IPC al dataset original
df = df.join(df_inflacion.select(['periodo','deflactor']), left_on="foto_mes", right_on="periodo", how="left")

# Dividimos cada una por el deflactor
df = df.with_columns([
    (pl.col(col) / pl.col("deflactor")).alias(f"{col}{sufijo}") 
    for col in monetarias
])

df = df.drop(["deflactor"])

In [13]:
df.write_parquet(dataset_path + output_file)
print(f"Dataset ajustado exportado como {dataset_path + output_file}")

Dataset ajustado exportado como C:/Eugenio/Maestria/DMEyF/datasets/competencia_02_inflacion_adj.parquet
